In [1]:
!pip install pokerkit

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
from math import inf
from tqdm import tqdm
from pokerkit import Automation, Mode, NoLimitTexasHoldem

#https://pokerkit.readthedocs.io/en/0.4/simulation.html

ModuleNotFoundError: No module named 'tqdm'

**Simulator Functions**

In [ ]:
#make a game
def make_samplegame(number_of_player):
  #Creating game template
  sample_game = NoLimitTexasHoldem(
  #positions : zeroth player will always be the small blind,
    #the first player will always be the big blind
      (
          Automation.ANTE_POSTING,
          Automation.BET_COLLECTION,
          Automation.BLIND_OR_STRADDLE_POSTING,
          Automation.HAND_KILLING,
          Automation.CHIPS_PUSHING,
          Automation.CHIPS_PULLING,
      ),
      True,  # Uniform antes? (False for big blind ante)
      1,     # antes
      (0,0),  # Blinds or straddles
      1,      # Min-bet
  )
  return sample_game(
      2, #starting stacks
      number_of_player,
  )
def sample_betting(board):
  while board.can_check_or_call():
    board.check_or_call()
def deal_sample_board(board, cards):
  if(board.can_burn_card()):
    board.burn_card("??") #burn a random card which not affect what we can deal next
  while(board.can_deal_board()):
    if cards != None:
      board.deal_board(cards) #initialize flop
    else:
      board.deal_board("??") #randomly deal board
def show_down(board):
  for i in range(5):
    if board.board_cards[i][0].unknown_status:
      board.board_cards[i][0] = board.deck_cards.popleft()
  while board.showdown_index != None:
    i = board.showdown_index
    hand1, hand2 = board.hole_cards[i]
    if hand1.unknown_status == True : #if the hand is [??, ??]
      hand1 = board.deck_cards.popleft() #draw two cards from the deck
      hand2 = board.deck_cards.popleft()
    board.show_or_muck_hole_cards((hand1,hand2), player_index= i) #reveal


###Simple Simulator of Poker

In [ ]:
def poke_simulator(number_of_player, myhands = None, _flop = None, _turn = None, _river = None):
  """
  This is a function that simulates the poker game.
    Args:
        number_of_player: Integer of the number of players in a given game.
        myhands: String of the hands dealt out to the player.
        _flop: String of the current face up cards at the board at flop
        _turn: String of the face up card added to the board at turn
        _river: String of the face up card added to the board at river

    Returns:
        1 if we win the game
  """
  #Pre-flop: fill Holes
  board = make_samplegame(number_of_player) # game state object
  board.verify_hole_dealing() # (card, player index)
  my_index = 2 #player position

  #hands
  for i in range(number_of_player):
    assert board.can_deal_hole()
    if i == my_index and myhands != None:
        board.deal_hole(myhands) #deal my cards
    else:
      board.deal_hole("????") #deal opp hands: two random cards
  #every round there is betting
  sample_betting(board)
  #flop, turn, and river
  deal_sample_board(board,_flop)
  sample_betting(board)
  deal_sample_board(board,_turn)
  sample_betting(board)
  deal_sample_board(board,_river)
  #last round (river) betting:
  sample_betting(board)
  #show_down and payoffs
  show_down(board)
  if board.payoffs[my_index] > 0:
    return 1
  else:
    return 0

In [ ]:
def PW_E(numb_player, hand, flop = None, turn = None, river = None):
  """
  This is a function that finds the probability of winning.
    Args:
        number_of_player: Integer of the number of players in a given game.
        myhands: String of the hands dealt out to the player.
        _flop: String of the current face up cards at the board at flop
        _turn: String of the face up card added to the board at turn
        _river: String of the face up card added to the board at river

    Returns:
        1 if we win the game
  """
  num_of_games = 200
  count = 0
  for i in (range(num_of_games)):
    count += poke_simulator(numb_player, hand, flop, turn, river)
  return count / num_of_games

#Agent

def our_agent(numb_player, hand, flop = None, turn = None, river = None):
  cpt = PW_E(numb_player, hand, flop, turn, river)
  if cpt > 0.15:
    return "CHECK"
  else:
    return "FOLD"

In [ ]:
from math import inf
from tqdm import tqdm
from pokerkit import Automation, Mode, NoLimitTexasHoldem
import random

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

evaluation_metrics = {"TW": 0, "FW": 0, "TL": 0, "FL": 0}

#Game
class PokerGame:
  def __init__(self, number_of_player, seat, agent= None, show_log = False):
    assert seat < number_of_player
    self.seat = seat
    self.num_of_players = number_of_player
    self.player_to_act = False
    self.show_log = show_log
    self.state = self.__make_game(number_of_player)

  #Function to create the game template
  def __make_game(self, number_of_player):
    self.minBet = 1
    self.sample_game = NoLimitTexasHoldem(
    #positions : zeroth player will always be the small blind,
      #the first player will always be the big blind
        (
            Automation.ANTE_POSTING,
            Automation.BET_COLLECTION,
            Automation.BLIND_OR_STRADDLE_POSTING,
            Automation.HAND_KILLING,
            Automation.CHIPS_PUSHING,
            Automation.CHIPS_PULLING,
        ),
        True,   # Uniform antes? (False for big blind ante)
        1,      # antes (The minimum bet every play contribute at the start of the game)
        (0,0),  # Blinds or straddles
        self.minBet,      # Min-bet
    )
    return self.sample_game(
        inf, #starting stacks
        number_of_player,
    )

  def start_round(self, agent_function = None, hands = None, flop = None, turn = None, river = None):
      # Initialize Game
      self.state = self.sample_game(
          inf, #starting stacks
          self.num_of_players,
      )
      self.Fold = False
      self.deal_board_state = 0
      if self.show_log:
          print(f"Your Seat is {self.seat}")

      # Preflop
      self.__deal_hands(hands)
      if self.show_log:
          print(f"Your hands are {self.state.hole_cards[self.seat]}")
      self.__betting()
      self.__player_action(agent_function)

      # Postflop, Turn, River
      board_cards = [flop, turn, river]
      while self.state.showdown_index is None:  # Until showdown
          if self.show_log:
              print("Dealt Card(s)")
          self.__deal_board(board_cards)
          self.deal_board_state += 1
          if self.show_log:
              print(f"Board: {self.state.board_cards}")
          self.__betting()
          self.__player_action(agent_function)

      # Showdown
      self.__show_down()

      # Evaluate Outcome
      actual_win = self.payoffs() > 0  # True if player actually won
      predicted_win = not self.Fold  # True if player did NOT fold

      if actual_win and predicted_win:
          result = "TW"  # True Positive
      elif not actual_win and predicted_win:
          result = "FW"  # False Positive
      elif not actual_win and not predicted_win:
          result = "TL"  # True Negative
      else:  # actual_win and not predicted_win
          result = "FL"  # False Negative

      return result


    #Evaluate
      if self.Fold == False:  # Player played
          if self.payoffs() > 0:
              update_evaluation_metrics("TP")  # True Positive
              return "TW"
          else:
              update_evaluation_metrics("FP")  # False Positive
              return "FP"
      else:  # Player folded
          if self.payoffs() > 0:
              update_evaluation_metrics("FN")  # False Negative
              return "FW"
          else:
              update_evaluation_metrics("TN")  # True Negative
              return "TN"

      if self.show_log:
        print(f"You Win? Paysoff : {self.payoffs()}")


  def can_check_or_call(self):
    return self.state.can_check_or_call()
  def can_bet_or_raise_to(self):
    return self.state.can_complete_bet_or_raise_to()
  def bet_or_raise_to(self, amount = None):
    return self.state.complete_bet_or_raise_to(amount or self.minBet)
  def payoffs(self):
    return self.state.payoffs[self.seat]

  """
  Evualte Player Action:
    Given by P(W|E), E= hands, flop, turn, river.
  """
  def __player_action(self, agent_function):
    assert self.player_to_act

    if self.Fold: # Plater already fold, no action allow
      self.state.check_or_call()
      self.player_to_act = False
      self.__betting() #Pass it to next player
      return

    while self.player_to_act:
      hand, flop, turn, river, = self.get_board_status()
      #Do player action
      action_list= ['Check', 'Call', 'Bet', 'Look', 'Fold']
      if agent_function != None:
        action = agent_function(self.num_of_players, hand, flop, turn, river)
      else:
        #default
        action = random.choice(action_list)
        action = action.upper().strip().strip('\n')

      if action == "LOOK":
        if self.show_log:
          print(f"Board : {self.state.board_cards}\n \
          Your Hands: {self.state.hole_cards[self.seat]}")
      elif action == "FOLD":
        if self.show_log:
          print("You Fold")

        """Save the Bet amont
        and Other info to save"""
        self.Fold = True

        self.state.check_or_call()
        self.player_to_act = False
        self.__betting() #Pass it to next player
        #We need to evaluate our hand at the end
      elif action == "CHECK" or action == "CALL":
        self.state.check_or_call()
        self.player_to_act = False
        self.__betting() #Pass it to next player
      elif action == "BET":
        amount = 90
        self.bet_or_raise_to(amount)
        self.player_to_act = False
        self.__betting() #Pass it to next player
      #End of player action

#---------------------------------------
  def get_board_status(self):
    return self.getHands(), self.getFlop(), self.getTurn(), self.getRiver()
  def getHands(self):
    return self.state.hole_cards[self.seat]
  def getFlop(self):
    board = self.state.board_cards
    if len(board) < 3:
      return None
    flop = board[0:3]
    return [flop[0][0], flop[1][0], flop[2][0]]
  def getTurn(self):
    board = self.state.board_cards
    if len(board) < 4:
      return None
    return board[3]
  def getRiver(self):
    board = self.state.board_cards
    if len(board) < 5:
      return None
    return board[4]

  #Hidden Game Process (For controling AI moement : modify _betting)
  def __betting(self):
    while self.state.can_check_or_call():
      if self.state.actor_index == self.seat:
        self.player_to_act = True
        return
      #opp always check or call
      self.state.check_or_call()
  def __deal_hands(self,hands):
    while self.state.can_deal_hole():
      if self.state.hole_dealee_index == self.seat:
        if(hands != None):
          self.state.deal_hole(hands)
        else:
          self.state.deal_hole() #deal my cards
      else:
        self.state.deal_hole("????") #deal opp hands: two random cards


  def __deal_board(self, given_boards):
    assert self.state.can_burn_card()
    self.state.burn_card("??") #burn a random card which not affect what we can deal next
    while(self.state.can_deal_board()):
      if given_boards[self.deal_board_state] != None:
        self.state.deal_board(given_boards[self.deal_board_state])
      else:
        self.state.deal_board() #randomly deal board
  def __show_down(self):
    #If the board_cards is unknown ??
    for i in range(len(self.state.board_cards)):
      if self.state.board_cards[i][0].unknown_status:
        self.state.board_cards[i][0] = self.state.deck_cards.popleft()
    while self.state.showdown_index != None:
      i = self.state.showdown_index
      hand1, hand2 = self.state.hole_cards[i]
      if hand1.unknown_status == True : #if the hand is [??, ??]
        hand1 = self.state.deck_cards.popleft() #draw two cards from the deck
        hand2 = self.state.deck_cards.popleft()
      self.state.show_or_muck_hole_cards((hand1,hand2), player_index= i) #reveal

In [ ]:
import pandas as pd
import ast

poker_df = pd.read_csv('poker_data.csv', on_bad_lines= 'skip')

# Function convert string representation of a list, while preserving NaN values
def convert(x):
    if pd.isna(x):
        return x
    if isinstance(x, str):
        return ''.join(ast.literal_eval(x))
    return x

# Apply conversion to relevant columns
columns_to_fix = ['hands', 'flop', 'turn', 'river']
for col in columns_to_fix:
    poker_df[col] = poker_df[col].apply(convert)
    poker_df[col] = poker_df[col].str.replace(r'1(?!0)', 'A', regex=True)
    poker_df[col] = poker_df[col].str.replace(r'10', 'T', regex=True)

# Display the modified DataFrame
poker_df.head()

<ipython-input-46-81320524bcf0>:4: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  poker_df = pd.read_csv('poker_data.csv', on_bad_lines= 'skip')


,hands,flop,turn,river,pot_size,your_chips
0,AhQh,Td6h4h,8d,2c,21.0,0
1,Ah9s,AcKs7d,Kd,NaN,11.0,0
2,Kc3c,JcJh4s,Ts,8c,31.0,0
3,Kh2h,9s7s4c,Ah,8c,16.0,0
4,KsKc,Td6h4h,8d,NaN,167.0,0


In [ ]:
def sim_games(row):
  """
    Function to simulate the games given a certain row

    Args:
    row: A given row in the dataset

    Returns:
    result: The result of the game (True Win, True Loss, False Win, False Lose)
  """
  flop= row['flop'] if not pd.isna(row['flop']) else None
  turn= row['turn'] if not pd.isna(row['turn']) else None
  river= row['river'] if not pd.isna(row['river']) else None
  result= pk.start_round(our_agent, row['hands'],flop, turn, river)
  return result

def find_accuracy(df):
  """
    Function to calculate the accuracy of the model

    Args:
    df: Our given Poker dataset

    Returns:
    accuracy: Calculated accuracy by (TL + TW)/(TL+ TW + FL + FW)
  """
  TL_count= (df['win']== 'TL').count()
  TW_count= (df['win']== 'TW').count()
  FL_count= (df['win']== 'FL').count()
  FW_count= (df['win']== 'FW').count()

  accuracy= (TL_count + TW_count)/ (TL_count + TW_count + FL_count + FW_count)
  return accuracy

sample= poker_df.sample(200, random_state=1)
sample['win']= sample.apply(sim_games, axis=1)
accuracy= find_accuracy(sample)

In [ ]:
accuracy

0.5

In [ ]:
def random_agent(num_players, hand, flop=None, turn=None, river=None):
    # Define possible actions
    actions = ["CHECK", "FOLD"]  

    # Choose a random action
    return random.choice(actions)

In [ ]:
# prompt: import random and play the poker game that was coded randomly and print back the result of how many times you win the game

from math import inf
from tqdm import tqdm
from pokerkit import Automation, Mode, NoLimitTexasHoldem
import pandas as pd
import ast
import random
import warnings

def run_random_simulation(num_games=100):
    true_win_count = 0
    tfold_win_count = 0
    rtrue_win_count = 0
    rtfold_win_count = 0
    pk = PokerGame(6, 0)  # Initialize the poker game 

    for _ in range(num_games):
      game = pk.start_round(random_agent)
      game2 = pk.start_round(our_agent)
      if game == "TL":
            rtfold_win_count += 1
      if game == "TW":
            rtrue_win_count += 1
      if game2 == "TL":
            tfold_win_count += 1
      if game2 == "TW":
            true_win_count += 1

    print(f"Number of fold wins out of {num_games}: {tfold_win_count}")
    print(f"Win rate: {tfold_win_count / num_games}")
    print(f"Number of wins out of {num_games}: {true_win_count}")
    print(f"Win rate: {true_win_count / num_games}")
    print(f"Random Number of fold wins out of {num_games}: {rtfold_win_count}")
    print(f"Win rate: {rtfold_win_count / num_games}")
    print(f"Random Number of wins out of {num_games}: {rtrue_win_count}")
    print(f"Win rate: {rtrue_win_count / num_games}")

run_random_simulation() # Call the function to start the simulation